# spotify songs popularity prediction models

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("../dataset/spotify.csv")
data.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [4]:
data.shape

(114000, 21)

In [5]:
data.columns.to_list()

['Unnamed: 0',
 'track_id',
 'artists',
 'album_name',
 'track_name',
 'popularity',
 'duration_ms',
 'explicit',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'track_genre']

In [6]:
data = data.drop(columns=["Unnamed: 0", "track_id"])
data.sample(10)

,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
28707,Zeds Dead;ILLENIUM,Where The Wild Things Are,Where The Wild Things Are,50,299240,False,0.344,0.672,6,-5.888,0,0.0586,0.008770,0.000001,0.1960,0.1330,164.197,4,dub
85710,Jaksi Taksi;Harlej,OffLine,Show,36,218320,False,0.324,0.930,11,-7.124,1,0.1020,0.000041,0.000097,0.1400,0.3580,180.223,4,punk-rock
35149,Calcinha Preta,"Eu Amo Demais, Vol. 21 (Ao Vivo)",Liga pra Mim (Ao Vivo),48,200773,False,0.513,0.779,4,-6.644,1,0.0362,0.163000,0.000012,0.6980,0.6180,163.980,4,forro
7579,The Devil Makes Three,Do Wrong Right,Car Wreck,24,425506,False,0.353,0.403,4,-10.813,1,0.0385,0.401000,0.308000,0.2360,0.3260,140.744,4,bluegrass
73669,Chloé (Thévenin),Mars 500,Mars 500,44,412257,False,0.756,0.676,11,-13.684,0,0.0459,0.011100,0.907000,0.0987,0.0883,123.007,4,minimal-techno
89845,El BAI;Marcianeke;King Savagge;Drakomafia;Vict...,Sonando Flaite (Remix),Sonando Flaite - Remix,42,410500,False,0.824,0.828,11,-5.040,0,0.2670,0.193000,0.000000,0.1190,0.7070,90.031,4,reggaeton
273,Kina Grannis,Yellow,Yellow,52,193536,False,0.623,0.132,0,-13.668,1,0.0350,0.893000,0.000000,0.2170,0.2660,82.105,4,acoustic
12721,Kary Ng,霏凡之選,逼得太緊,21,284000,False,0.527,0.539,3,-5.331,1,0.0277,0.581000,0.000000,0.1540,0.2290,126.885,4,cantopop
78605,Glitzer Gischi,Rumpop'n,Rumpop'n,26,166521,True,0.677,0.960,6,-4.062,1,0.1750,0.076100,0.000000,0.3470,0.7430,137.868,4,party
21603,NSG;Tion Wayne,Roots,Options,56,240081,True,0.836,0.621,1,-4.684,0,0.0894,0.389000,0.000092,0.1040,0.7620,101.993,4,dancehall


In [7]:
data.shape

(114000, 19)

In [9]:
features = data.columns.to_list()
features

['artists',
 'album_name',
 'track_name',
 'popularity',
 'duration_ms',
 'explicit',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'track_genre']